In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.3.0'

**Tensorflow 1** == define and run  
**Tensorflow 2** == similar to **define by run**  
because version.2 has no **Session** function  
or using [tf.compat.v1.Session](https://www.tensorflow.org/api_docs/python/tf/compat/v1/Session#used-in-the-notebooks)

In [3]:
# constant
hello = tf.constant('hello, tensorflow')
print(hello)

a,b = tf.constant(10), tf.constant(20)
c = tf.add(a,b)
print(c)

tf.Tensor(b'hello, tensorflow', shape=(), dtype=string)
tf.Tensor(30, shape=(), dtype=int32)


**1. One Feature Node**  
out = wx + b

In [4]:
class One_Feature_Node:
    def __init__(self):
        self.w = tf.Variable([[0.1]])
        self.b = tf.Variable([[0.5]])
        
    def __call__(self, x):
        return self.get_output(x)
    
    def get_output(self,x):
        out = tf.matmul(x,self.w) # matrix multiplexer
        print("matmul result: ", out.numpy())
        out = tf.add(out, self.b)
        print("adding bias result: ", out.numpy())
        out = tf.math.sigmoid(out)
        print("sigmoid result:", out.numpy())
        return out

In [5]:
# Single Input
x = tf.constant([[1.0]])

one_feature_node = One_Feature_Node()
one_feature_node(x)

matmul result:  [[0.1]]
adding bias result:  [[0.6]]
sigmoid result: [[0.6456563]]


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.6456563]], dtype=float32)>

**2. Two Feartures Node**  
Input: 1x2  
w: 2x1  
output: 1x2_2x1 = 1x1  

In [6]:
class Two_Features_Node:
    def __init__(self):
        self.w = tf.Variable([[0.1], [0.2]])
        self.b = tf.Variable([[0.5]])
        
    def __call__(self, x):
        return self.get_output(x)
    
    def get_output(self,x):
        out = tf.matmul(x,self.w)
        print("matmul result: ", out.numpy())
        out = tf.add(out, self.b)
        print("adding bias result: ", out.numpy())
        out = tf.math.sigmoid(out)
        print("sigmoid result:", out.numpy())
        return out

In [7]:
# Multi Features Single Input (Two features for one data)
x = tf.constant([[1.0,2.0]])

two_features_node = Two_Features_Node()
two_features_node(x)

matmul result:  [[0.5]]
adding bias result:  [[1.]]
sigmoid result: [[0.7310586]]


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.7310586]], dtype=float32)>

**3. N inputs for Two Features Node**

In [8]:
# Multi Input Data (3 samples)
x = tf.constant([[1.0,2.0],[2.0,3.0],[4.0,5.0]])

two_features_node = Two_Features_Node()
two_features_node(x)

matmul result:  [[0.5]
 [0.8]
 [1.4]]
adding bias result:  [[1. ]
 [1.3]
 [1.9]]
sigmoid result: [[0.7310586]
 [0.7858349]
 [0.8698915]]


<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[0.7310586],
       [0.7858349],
       [0.8698915]], dtype=float32)>

**4. using autograph in TF2**

In [9]:
import timeit # check the more faster execution

In [10]:
class Two_Features_Node2:
    def __init__(self):
        self.w = tf.Variable([[0.1], [0.2]])
        self.b = tf.Variable([[0.5]])
        
    def __call__(self, x):
        return self.get_output(x)
    
    # running on tpu
    def get_output(self,x):
        for i in range(50):
            out = tf.matmul(x,self.w)
            out = tf.add(out, self.b)
            out = tf.math.sigmoid(out)
        return out
    
    # running on gpu
    @tf.function # tf2 autograph
    def get_output_faster(self,x):
        for i in range(50):
            out = tf.matmul(x,self.w)
            out = tf.add(out, self.b)
            out = tf.math.sigmoid(out)
        return out

In [11]:
two_features_node2 = Two_Features_Node2()
get_output_without_tf_function_timeit = timeit.timeit(lambda: two_features_node2.get_output(x), number=300)
get_output_faster_with_tf_function_timeit = timeit.timeit(lambda: two_features_node2.get_output_faster(x), number=300)
time_diff = round(get_output_without_tf_function_timeit / get_output_faster_with_tf_function_timeit, 1)

In [12]:
print("get_output_faster(@tf.function) is " + str(time_diff) + " times faster than get_output_without_tf_function!")

get_output_faster(@tf.function) is 2.5 times faster than get_output_without_tf_function!
